# Importing Runkeeper Data

This script imports my training data from Runkeeper API.

### Step 1 - Loading required libraries

In [336]:
import datetime, json, urllib2

import numpy as np
import pandas as pd

from pandas import Series, DataFrame

### Step 2 - Importing data into a Pandas DataFrame

In [337]:
url = "https://api.runkeeper.com/fitnessActivities?pageSize=300&access_token=0006f0cb9cd840d1894b0f5dbcb7c010"
data = urllib2.urlopen(url).read()
data = json.loads(data)
data = data["items"]

df = pd.io.json.json_normalize(data) # The DataFrame containing the training raw data.

# Now let's cleaning the table. I just need three information from my exercises: distance, duration and time.

df = df[["duration", "start_time", "total_distance"]]
df.columns = ["duration", "start", "distance"]

# Let's get a look into the table

df.head()

,duration,start,distance
0,2003,"Wed, 6 Jan 2016 17:05:31",6048.640242
1,908,"Mon, 4 Jan 2016 16:34:34",3050.470102
2,1681,"Sun, 3 Jan 2016 16:29:20",5023.396811
3,304,"Wed, 30 Dec 2015 09:07:12",1013.794585
4,1673,"Wed, 30 Dec 2015 08:18:22",5023.939766


### Step 3 - Converting data values

In [338]:
# Converting start date string to datetime format 
def to_datetime(string):
    return datetime.datetime.strptime(string, "%a, %d %b %Y %X")

df.start = df.start.apply(to_datetime)


# Now converting duration string to timedelta format
def to_timedelta(num):
    return pd.Timedelta(str(num) + "seconds")

df.duration = df.duration.apply(to_timedelta)


# And finnaly, let's round distance to meters and convert it to Km
def to_km(num):
    return int(num) / 1000.0

df.distance = df.distance.apply(to_km)

# Let's look again:

df.head()

# Seems better now :)

,duration,start,distance
0,00:33:23,2016-01-06 17:05:31,6.048
1,00:15:08,2016-01-04 16:34:34,3.050
2,00:28:01,2016-01-03 16:29:20,5.023
3,00:05:04,2015-12-30 09:07:12,1.013
4,00:27:53,2015-12-30 08:18:22,5.023
